# 实现 Decoder Only 架构的 Transformer 模型

Decoder Only 架构是 Transformer 模型的一种变体，**仅保留 Transformer 中的解码器（Decoder）部分**，并对其进行调整和优化。这种架构在自然语言处理（NLP）领域被广泛用于生成式任务（如文本生成、对话系统等），典型代表是 GPT 系列模型（如 GPT-3、ChatGPT）。以下是基于 Transformer 变体架构的对比表：

| 架构类型            |    典型模型    | 适配任务 | 与大模型的关系             |
|:----------------|:----------:|:-------:|:--------------------|
| Decoder Only    | GPT、LLaMA  | 生成任务（文本生成、对话） | 主流大模型的首选架构          |
| Encoder Only    |    BERT    | 理解任务（分类、NER） | 参数量通常较小，不适合生成任务     |
| Encoder-Decoder |  T5、BART   | 序列到序列任务（翻译、摘要）| 参数量较大但复杂度高，大模型中较少采用 |

## 详细结构图

![Decoder Only Arch](img/Decoder-Only-Arch.svg)

---

## 代码实现

---

In [ ]:
from transformer_decoder import *

class DecoderOnlyTransformer(nn.Module):
    def __init__(
            self,
            layer_num,
            vocab_size,
            dim_emb,
            dim_head,
            head_num,
            max_seq_len:int = 50000,
            training:bool = False
    ):
        """
        Decoder Only Transformer

        :param layer_num: 层数
        :param vocab_size: 词库大小
        :param dim_emb: 嵌入维度大小
        :param dim_head: 单个注意力模块的头维度
        :param head_num: 注意力头数
        :param max_seq_len: 最大序列长度
        :param training: 是否开启训练模式
        """
        super().__init__()

        self.layer_num = layer_num
        self.vocab_size = vocab_size
        self.dim_emb = dim_emb
        self.dim_head = dim_head
        self.head_num = head_num
        self.max_seq_len = max_seq_len
        self.training = training

        # 位置编码模块
        self.position_embedding = PositionalEmbedding(dim_emb, max_len = max_seq_len)

        # 创建词嵌入层
        self.word_embedding = nn.Embedding(vocab_size, dim_emb)

        # 创建解码器
        self.decoders = nn.ModuleList()
        for i in range(self.layer_num):
            self.decoders.append(Decoder(dim_emb, dim_head, head_num))

        # 创建线性层，线性层的作用是将嵌入维度转换为词表维度
        self.linear = nn.Linear(dim_emb, vocab_size)

    def forward(self, x, mask = None, dec_kv_caches = None):

        batch, seq_len = x.size()
        training = self.training

        # 输出词向量嵌入
        embedded_x = self.word_embedding(x)

        # 计算解码器掩码矩阵
        if seq_len > 1:
            triu_mask_mat = torch.triu(
                torch.ones(seq_len, seq_len, dtype=torch.bool, device=x.device),
                diagonal=1
            )
            if mask is not None:
                dec_mask_mat = torch.stack([triu_mask_mat | (~ mask[i].bool()) for i in range (batch)]).type(dtype=torch.bool)
                dec_mask_mat = dec_mask_mat.view(batch, 1, seq_len, seq_len).requires_grad_(False)
            else:
                dec_mask_mat = triu_mask_mat.requires_grad_(False)
        else:
            dec_mask_mat = None

        # 检查是否需要缓存
        new_dec_kv_caches = [] if not training else None
        if training:
            # 位置编码
            pe_x = self.position_embedding(embedded_x, 0, seq_len)

            # 计算解码器的计算结果，其最终输出维度为 (batch, seq_len, dim_emb)
            for decoder in self.decoder_blocks:
                pe_x, _ = decoder(
                    pe_x, pe_x, pe_x,
                    dec_mask_mat = dec_mask_mat
                )
        else:
            # 位置编码
            if dec_kv_caches is not None:
                # dec_kv_caches 对应的索引： [编码器层数][编码器中的多头注意力的层数][K Cache/V Cache]
                # K Cache / V Cache 对应的张量维度： (批数, 头数, 缓存的 K / V 序列长度, 头维度)
                cached_seq_len = dec_kv_caches[0][0][0].size(2)
                pe_x = self.position_embedding(embedded_x, cached_seq_len, cached_seq_len + 1)
            else:
                pe_x = self.position_embedding(embedded_x, 0, seq_len)

            # 计算解码器的计算结果，并缓存 KV 的值
            for dec_idx, decoder in enumerate(self.decoders):
                if dec_kv_caches is not None:
                    kv_caches = dec_kv_caches[dec_idx]
                else:
                    kv_caches = None

                pe_x, new_kv_caches = decoder(
                    pe_x, pe_x, pe_x,
                    dec_mask_mat = dec_mask_mat,
                    use_cache = True,
                    kv_caches = kv_caches
                )
                new_dec_kv_caches.append(new_kv_caches)

        # 最终输出维度为 (batch, output_seq_len, vocab_size)
        logits = self.linear(pe_x)
        if not training:
            output = F.softmax(logits, dim=-1)
            return output, new_dec_kv_caches
        else:
            # 需要使用交叉熵计算损失，而 pytorch 提供的交叉熵算法内部已经包含了 softmax
            # 所以这里结果不要进行 softmax ，否则会导致损失计算不稳定
            return logits

## 模型测试

---

In [ ]:
import random

# 配置
# =====================================
layer_num = 6    # 编码器数量
vocab_size = 256 # 词表大小
dim_emb = 128    # 词向量维度
dim_head = 32    # 注意力头维度
head_num = 4     # 注意力头数

batch = 3        # 测试数据批数
seq_len = 12     # 测试输入数据长度
# =====================================

# 创建 transformer 网络
transformer = DecoderOnlyTransformer(layer_num, vocab_size, dim_emb, dim_head, head_num)

def rand_seq_code(batch, seq_len):
    """
    随机序列编码

    :param batch: 批数
    :param seq_len: 序列长度
    :return: 随机序列编码
    """

    data = []
    for i in range(batch):
        bat_data = [random.randint(0, vocab_size) for _ in range(seq_len)]
        data.append(bat_data)
    return torch.tensor(data)

# 构建随机输入输出序列
x = rand_seq_code(batch, seq_len)
mask = torch.stack([torch.arange(0,seq_len) for _ in range(batch)]) < 8

# ===============
#  模拟第一次调用
# ===============

# transformer 输出
output, kv_caches = transformer(
    x, mask
)

# 打印输出结果
print(f'output size  : {output.size()}')
print(f'kv cache len : {len(kv_caches)}')

# 预测下一个值
token = torch.argmax(output[:, -1, :], dim=-1)
print(f'prediction token : {token}')

# ===============
#  模拟第二次调用
# ===============

# transformer 输出
output_next, kv_caches_next = transformer(
    x = token.view(batch, 1),
    dec_kv_caches = kv_caches
)
token_next = torch.argmax(output_next[:, -1, :], dim=-1)
print(f'next prediction token : {token_next}')